In [1]:
#install.packages('/users/soumyak/QuASAR_0.1.tar.gz')
library('QuASAR')

In [ ]:
indirs <- list.dirs('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/allelic_imbalance/quasar/input', recursive=FALSE)
for (dir in indirs) {
    patient <- gsub(".*/", "", dir)
    print(patient)
    output.names <- gsub(".quasar.in.gz", "", list.files(dir))
    print(output.names)
    fileNames <- paste0(dir, '/', list.files(dir))
    ase.dat <- UnionExtractFields(fileNames, combine=TRUE)
    ase.dat.gt <- PrepForGenotyping(ase.dat, min.coverage=5)
    #str(ase.dat.gt)
    sample.names <- colnames(ase.dat.gt$ref)
    ase.joint <- fitAseNullMulti(ase.dat.gt$ref, ase.dat.gt$alt, log.gmat=log(ase.dat.gt$gmat))
    #str(ase.joint)
    #head(ase.joint$gt)
    out_dat <- data.frame(ase.dat.gt$annotations[, -5], map=ase.joint$gt)
    outfile <- paste0('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/allelic_imbalance/quasar/output/', patient, '/genotypes.txt')
    write.table(out_dat, file=outfile, row.names=FALSE, col.names=TRUE, quote=FALSE, sep="\t")
    print(outfile)
    ourInferenceData <- aseInference(gts=ase.joint$gt, eps.vect=ase.joint$eps, priors=ase.dat.gt$gmat, ref.mat=ase.dat.gt$ref, alt.mat=ase.dat.gt$alt, min.cov=10, sample.names=sample.names, annos=ase.dat.gt$annotations)
    for (i in seq_along(ourInferenceData)) {
        outfile <- paste0('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/allelic_imbalance/quasar/output/', patient, '/', output.names[i], '.inference.txt')
        print(outfile)
        #print(ourInferenceData[[i]]$dat[order(ourInferenceData[[i]]$dat$pval2.het.ind.),])
        write.table(ourInferenceData[[i]]$dat[order(ourInferenceData[[i]]$dat$pval2.het.ind.),], file=outfile, row.names=FALSE, col.names=TRUE, quote=FALSE, sep="\t")
    }
}